# Modelos del lenguaje con RNNs



## 1. Carga y procesado del texto

In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
import sys
import random
import io

path = keras.utils.get_file(
    fname="don_quijote.txt",
    origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io"
)


**1.1.** Leer todo el contenido del texto en una única variable ***text*** y convertir el string a minúsculas

In [2]:
with open(path, encoding="utf8") as f:
  text = f.read().lower()

In [3]:
len(text)

2071198

Puede comprobar que se ha realizado la variación y que el texto se encuentra en minuscula.

In [4]:
#Se muestran los primeros 500 carácteres del texto y obtenemos la longitud del texto utilizando "len(text)".
print("Longitud del texto: {}".format(len(text)))
print(text[0:500])

Longitud del texto: 2071198
capítulo primero. que trata de la condición y ejercicio del famoso hidalgo
don quijote de la mancha


en un lugar de la mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor. una olla de algo más vaca que carnero,
salpicón las más noches, duelos y quebrantos los sábados, lantejas los
viernes, algún palomino de añadidura los domingos, consumían las tres
partes de su hacienda. el resto della co


## 2. Procesado de los datos

#### 2.1. Obtención de los caracteres y mapas de caracteres




In [5]:
chars=sorted(list(set(text)))
char_indices = dict((c,i) for i, c in enumerate(chars))
indice_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
#indice_char
char_indices
len(chars)

61

#### 2.2. Obtención de secuencias de entrada y carácter a predecir

Ahora, se obtiene las secuencias de entrada en formato texto y los correspondientes caracteres a predecir. 

In [7]:
# Definia el tamaño de las secuencias. Puede dejar este valor por defecto.
SEQ_LENGTH = 35
step=3
rawX = []
rawy = []

for i in range(0, len(text) - SEQ_LENGTH, step):
    rawX.append(text[i: i+SEQ_LENGTH])
    rawy.append(text[i+SEQ_LENGTH])

Indicar el tamaño del training set que acabamos de generar.

In [8]:
n_sentences=len(rawX)
print("Tamaño del training set generado: ",n_sentences)

Tamaño del training set generado:  690388


Como el Quijote es muy largo y tiene muchas secuencias, se puede encontrar con problemas de memoria. Por ello, se elije un número máximo de ellas.

In [9]:
MAX_SEQUENCES = 100000

perm = np.random.permutation(len(rawX)) #Permutar aleatoriamente una secuencia, o devolver un rango permutado.
rawX, rawy = np.array(rawX), np.array(rawy)
rawX, rawy = rawX[perm], rawy[perm]
rawX, rawy = list(rawX[:MAX_SEQUENCES]), list(rawy[:MAX_SEQUENCES])

print(len(rawX))

100000


#### 2.3. Obtención de input X y output y para el modelo





In [10]:
X = np.zeros((len(rawX), SEQ_LENGTH , len(chars)))
y = np.zeros((len(rawX), len(chars)))

In [11]:
for i, sentence in enumerate(rawX):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[rawy[i]]] = 1

## 3. Definición del modelo y entrenamiento

Se Define el modelo que utiliza una **LSTM** con **128 unidades internas**. 

In [12]:
model= Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(chars))))
model.add(Dropout(0.5))
model.add(Dense(len(chars), activation= 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               97280     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 61)                7869      
                                                                 
Total params: 105149 (410.74 KB)
Trainable params: 105149 (410.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X, y, batch_size=32, epochs=15, verbose=1)

Epoch 1/15
3125/3125 [==============================] - 144s 45ms/step - loss: 2.4775 - accuracy: 0.2843
Epoch 2/15
3125/3125 [==============================] - 142s 46ms/step - loss: 2.1345 - accuracy: 0.3543
Epoch 3/15
3125/3125 [==============================] - 144s 46ms/step - loss: 2.0293 - accuracy: 0.3831
Epoch 4/15
3125/3125 [==============================] - 149s 48ms/step - loss: 1.9570 - accuracy: 0.4026
Epoch 5/15
3125/3125 [==============================] - 142s 45ms/step - loss: 1.9032 - accuracy: 0.4152
Epoch 6/15
3125/3125 [==============================] - 143s 46ms/step - loss: 1.8569 - accuracy: 0.4316
Epoch 7/15
3125/3125 [==============================] - 143s 46ms/step - loss: 1.8179 - accuracy: 0.4405
Epoch 8/15
3125/3125 [==============================] - 145s 46ms/step - loss: 1.7823 - accuracy: 0.4521
Epoch 9/15
3125/3125 [==============================] - 142s 45ms/step - loss: 1.7559 - accuracy: 0.4597
Epoch 10/15
3125/3125 [==============================] 

Para ver cómo evoluciona SU modelo del lenguaje,  genere texto según va entrenando. Para ello, se programa una función que, utilizando el modelo en su estado actual, genera texto, con la idea de ver cómo se va generando texto al entrenar cada epoch.




In [15]:
def sample(probs, temperature=1.0):
    """Nos da el índice del elemento a elegir según la distribución
    de probabilidad dada por probs.

    Args:
      probs es la salida dada por una capa softmax:
        probs = model.predict(x_to_predict)[0]

      temperature es un parámetro que nos permite obtener mayor
        "diversidad" a la hora de obtener resultados.

        temperature = 1 nos da la distribución normal de softmax
        0 < temperature < 1 hace que el sampling sea más conservador,
          de modo que sampleamos cosas de las que estamos más seguros
        temperature > 1 hace que los samplings sean más atrevidos,
          eligiendo en más ocasiones clases con baja probabilidad.
          Con esto, tenemos mayor diversidad pero se cometen más
          errores.
    """
    # Cast a float64 por motivos numéricos
    probs = np.asarray(probs).astype('float64')

    # logaritmo de probabilidades y aplicamos reducción
    # por temperatura.
    probs = np.log(probs) / temperature

    # Volvemos a aplicar exponencial y normalizamos de nuevo
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)

    # Hacemos el sampling dadas las nuevas probabilidades
    # de salida (ver doc. de np.random.multinomial)
    samples = np.random.multinomial(1, probs, 1)
    return np.argmax(samples)


Utilizando la función anterior y el modelo entrenado, se añade un callback a al modelo para que, según vaya entrenando, se vean los valores que resultan de generar textos con distintas temperaturas al acabar cada epoch.



In [16]:
TEMPERATURES_TO_TRY = [0.2] #, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 300

def generate_text(seed_text, model, length=300, temperature=1, max_length=30):
    """Genera una secuencia de texto a partir de seed_text utilizando model.

    La secuencia tiene longitud length y el sampling se hace con la temperature
    definida.
    """

    # Aquí guardaremos nuestro texto generado, que incluirá el
    # texto origen
    generated = seed_text

    # Utilizar el modelo en un bucle de manera que generemos
    # carácter a carácter. Habrá que construir los valores de
    # X_pred de manera similar a como hemos hecho arriba, salvo que
    # aquí sólo se necesita una oración
    # Nótese que el x que utilicemos tiene que irse actualizando con
    # los caracteres que se van generando. La secuencia de entrada al
    # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
    # incluya el último caracter predicho.

    prediction = []

    for i in range(length):
        # Make numpy array to hold seed
        X = np.zeros((1, len(generated), len(chars) ))

        # Set one-hot vectors for seed sequence
        for t, char in enumerate(seed_text):
            X[0, t, char_indices[char]] = 1

        # Generate prediction for next character
        preds = model.predict(X, verbose=0)[0]
        # Choose a character from the prediction probabilities
        next_index = sample(preds,0.2)
        next_char = indice_char[next_index]

        prediction.append(next_char)
        # Add the predicted character to the seed sequence so the next prediction
        # includes this character in it's seed.
        #generated += next_char
        seed_text = seed_text[1:] + next_char

        print(next_char, end= " ");
        # Flush so we can see the prediction as it's generated
        sys.stdout.flush()

    prediction = ''.join(prediction)
    sys.stdout.flush()

    return generated


def on_epoch_end(epoch, logs):
  print("\n\n\n")

  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
  seed_text = text[start_pos:start_pos + SEQ_LENGTH]
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))

    generated_text = generate_text(seed_text, model,
                                   GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))


generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [18]:


model.fit(X, y, batch_size=32, epochs=15, callbacks=generation_callback)

Epoch 1/15
3125/3125 [==============================] - ETA: 0s - loss: 1.6052 - accuracy: 0.5042



------> Epoch: 1 - Generando texto con temperature 0.2
n   p e n s a d o   d e   l a   m e r a d o ,   s i n o   e s t a b a   d e   s u   p a r t e   d e   l a   c a b a l l e r o   d e   l a   m e r a d a   d e   l a   m a n o ,   y   a   s u   s e ñ o r   d e   l a   m e r a d a   d e   s u   c a b a l l e r o   a n d a n t e   s e   h a b í a   d e   d e l   c o m p e r o   d e   s u   s e ñ o r   d e   l a   m e r a d a ,   d e   s u   m e r a d o   d e   l a   m e r a d   d e   l a s   c a b a l l e r o s   d e   l a   d e s t a   a l l a   d e   l a   m e r a d o ,   y   d e   m u c h a   d e   l a   m a l a   d e   s u   p e r s e n t e   Seed: , y, aunque tenía más cuartos que u
Texto generado: , y, aunque tenía más cuartos que u
3125/3125 [==============================] - 170s 54ms/step - loss: 1.6052 - accuracy: 0.5042
Epoch 2/15
3124/3125 [============================>.] - 

In [19]:
TEMPERATURES_TO_TRY = [0.5] #, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 300

def generate_text(seed_text, model, length=300, temperature=1, max_length=30):
    """Genera una secuencia de texto a partir de seed_text utilizando model.

    La secuencia tiene longitud length y el sampling se hace con la temperature
    definida.
    """

    # Aquí guardaremos nuestro texto generado, que incluirá el
    # texto origen
    generated = seed_text

    # Utilizar el modelo en un bucle de manera que generemos
    # carácter a carácter. Habrá que construir los valores de
    # X_pred de manera similar a como hemos hecho arriba, salvo que
    # aquí sólo se necesita una oración
    # Nótese que el x que utilicemos tiene que irse actualizando con
    # los caracteres que se van generando. La secuencia de entrada al
    # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
    # incluya el último caracter predicho.

    prediction = []

    for i in range(length):
        # Make numpy array to hold seed
        X = np.zeros((1, len(generated), len(chars) ))

        # Set one-hot vectors for seed sequence
        for t, char in enumerate(seed_text):
            X[0, t, char_indices[char]] = 1

        # Generate prediction for next character
        preds = model.predict(X, verbose=0)[0]
        # Choose a character from the prediction probabilities
        next_index = sample(preds,0.2)
        next_char = indice_char[next_index]

        prediction.append(next_char)
        # Add the predicted character to the seed sequence so the next prediction
        # includes this character in it's seed.
        #generated += next_char
        seed_text = seed_text[1:] + next_char

        print(next_char, end= " ");
        # Flush so we can see the prediction as it's generated
        sys.stdout.flush()

    prediction = ''.join(prediction)
    sys.stdout.flush()

    return generated


def on_epoch_end(epoch, logs):
  print("\n\n\n")

  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
  seed_text = text[start_pos:start_pos + SEQ_LENGTH]
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))

    generated_text = generate_text(seed_text, model,
                                   GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))


generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [20]:
model.fit(X, y, batch_size=32, epochs=1, callbacks=generation_callback)

3124/3125 [============================>.] - ETA: 0s - loss: 1.4526 - accuracy: 0.5445



------> Epoch: 1 - Generando texto con temperature 0.5
a   s u   p a r t e   d e   l a   m a n o   e l   c a b a l l e r o   d e   l a   m a n o   e n   e l   c u a l   c o m o   d e   l a   c o n t e n c i ó n   d e   s u   a m o   d e   l a   p a l l a   d e   l a   c a r t e ,   y   a   l o   q u e   l a s   c a b a l l e r o s   d e   l a   m e n t e   d e   l a   c a r a c i a   d e   l a   m a n o   e l   c a b a l l e r o   d e   l a   c a r a   y   d e   l a   c a r t e ,   y   s e   e s p e r a   d e   l a   c a r t e   a   l a   m e s t e   c a r a   d e   l a   c a r a c i a   d e   l a   c a b a l l e r í a   d e   s u   p a r t e ,   Seed:  paso que había
entrado, se volvió 
Texto generado:  paso que había
entrado, se volvió 
3125/3125 [==============================] - 174s 56ms/step - loss: 1.4525 - accuracy: 0.5445


In [21]:
TEMPERATURES_TO_TRY = [1.0] #, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 300

def generate_text(seed_text, model, length=300, temperature=1, max_length=30):
    """Genera una secuencia de texto a partir de seed_text utilizando model.

    La secuencia tiene longitud length y el sampling se hace con la temperature
    definida.
    """

    # Aquí guardaremos nuestro texto generado, que incluirá el
    # texto origen
    generated = seed_text

    # Utilizar el modelo en un bucle de manera que generemos
    # carácter a carácter. Habrá que construir los valores de
    # X_pred de manera similar a como hemos hecho arriba, salvo que
    # aquí sólo se necesita una oración
    # Nótese que el x que utilicemos tiene que irse actualizando con
    # los caracteres que se van generando. La secuencia de entrada al
    # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
    # incluya el último caracter predicho.

    prediction = []

    for i in range(length):
        # Make numpy array to hold seed
        X = np.zeros((1, len(generated), len(chars) ))

        # Set one-hot vectors for seed sequence
        for t, char in enumerate(seed_text):
            X[0, t, char_indices[char]] = 1

        # Generate prediction for next character
        preds = model.predict(X, verbose=0)[0]
        # Choose a character from the prediction probabilities
        next_index = sample(preds,0.2)
        next_char = indice_char[next_index]

        prediction.append(next_char)
        # Add the predicted character to the seed sequence so the next prediction
        # includes this character in it's seed.
        #generated += next_char
        seed_text = seed_text[1:] + next_char

        print(next_char, end= " ");
        # Flush so we can see the prediction as it's generated
        sys.stdout.flush()

    prediction = ''.join(prediction)
    sys.stdout.flush()


    return generated


def on_epoch_end(epoch, logs):
  print("\n\n\n")

  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
  seed_text = text[start_pos:start_pos + SEQ_LENGTH]
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))

    generated_text = generate_text(seed_text, model,
                                   GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))


generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [22]:
model.fit(X, y, batch_size=32, epochs=1, callbacks=generation_callback)

3125/3125 [==============================] - ETA: 0s - loss: 1.4514 - accuracy: 0.5462



------> Epoch: 1 - Generando texto con temperature 1.0
s ,   y   a   s u   s e ñ o r a   d e   l a   v e r d a d ,   y   a   l o   q u e   l a   s e ñ o r a   m u e s t r a   m e r c e d   d e   l a   m e n t e   c o n   e l l o s   s e   l a   c a b a l l e r o   a   l a s   c a r a l l a r   a   l a   m a n o   e l   c a b a l l e r o   d e   l a   v e r d a d ,   y   e s t a   d e   l a   c a r t e   d e   l a   m e r a d o   d e   l a   m e n t e   c o n   l a   c a r a   q u e   s e   m e   a c a b a r a   e s t a   m a n o   a   l a   m e s t e   e n   l a   m e n t e   d e   l a   m e r a d o   d e   l a   c a r t e   a   l a   m e r a d o Seed:  se le acabasen de salir los diente
Texto generado:  se le acabasen de salir los diente
3125/3125 [==============================] - 171s 55ms/step - loss: 1.4514 - accuracy: 0.5462


In [23]:
TEMPERATURES_TO_TRY = [1.2] #, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 300

def generate_text(seed_text, model, length=300, temperature=1, max_length=30):
    """Genera una secuencia de texto a partir de seed_text utilizando model.

    La secuencia tiene longitud length y el sampling se hace con la temperature
    definida.
    """

    # Aquí guardaremos nuestro texto generado, que incluirá el
    # texto origen
    generated = seed_text

    # Utilizar el modelo en un bucle de manera que generemos
    # carácter a carácter. Habrá que construir los valores de
    # X_pred de manera similar a como hemos hecho arriba, salvo que
    # aquí sólo se necesita una oración
    # Nótese que el x que utilicemos tiene que irse actualizando con
    # los caracteres que se van generando. La secuencia de entrada al
    # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
    # incluya el último caracter predicho.

    prediction = []

    for i in range(length):
        # Make numpy array to hold seed
        X = np.zeros((1, len(generated), len(chars) ))

        # Set one-hot vectors for seed sequence
        for t, char in enumerate(seed_text):
            X[0, t, char_indices[char]] = 1

        # Generate prediction for next character
        preds = model.predict(X, verbose=0)[0]
        # Choose a character from the prediction probabilities
        next_index = sample(preds,0.2)
        next_char = indice_char[next_index]

        prediction.append(next_char)
        # Add the predicted character to the seed sequence so the next prediction
        # includes this character in it's seed.
        #generated += next_char
        seed_text = seed_text[1:] + next_char

        print(next_char, end= " ");
        # Flush so we can see the prediction as it's generated
        sys.stdout.flush()

    prediction = ''.join(prediction)
    sys.stdout.flush()
    
    return generated


def on_epoch_end(epoch, logs):
  print("\n\n\n")

  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
  seed_text = text[start_pos:start_pos + SEQ_LENGTH]
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))

    generated_text = generate_text(seed_text, model,
                                   GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))


generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [24]:
model.fit(X, y, batch_size=32, epochs=1, callbacks=generation_callback)

3124/3125 [============================>.] - ETA: 0s - loss: 1.4420 - accuracy: 0.5495



------> Epoch: 1 - Generando texto con temperature 1.2
s   d e   l a   c a r t e   d e   l a   m a n e r a   a   s u   a m i g o   a   l a   m e n t e   d e   l a   m e n t e ,   y   a l   s u s   d e   l a   s e ñ o r a   d e   c u r a   q u e   s e   l e   h a b í a   d e   h a b e r   q u e   e s t a b a   l o s   d e   l a   m a n o   e n   e l   c u e l a   d e   l a   c a r t e   d e   l a   c a r t e   d e   l a   m i s m a   d e   l a   p r e s t e   a   l a   m e r a d o   d e   t o d a   l a   m e s t r a c i ó   d e   l a   c a b a l l e r o   a n d a n t e   s e   l e   h a b í a   d e   s e r   s u   h a b í a   d e   l a   c a r t e Seed:  toda la sala levantando los dosele
Texto generado:  toda la sala levantando los dosele
3125/3125 [==============================] - 177s 57ms/step - loss: 1.4420 - accuracy: 0.5495
